In [1]:
import xarray as xr
import numpy as np

# import hvplot.xarray
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
# xroms is an augmented xarray to work with ROMS outputs
import xroms
# cmocean colormaps for oceanography. xcmocean for auto choice of colormap
# import cmocean.cm as cmo
# import xcmocean 

In [25]:
url = "https://tds.marine.rutgers.edu/thredds/dodsC/roms/doppio/2017_da/avg/files/doppio_avg_20240201_0000_0001.nc"
ds = xr.open_dataset(url)
# ds.cf.describe()
ds

<xarray.Dataset>
Dimensions:         (tracer: 2, boundary: 4, s_rho: 40, s_w: 41, eta_rho: 106, xi_rho: 242, eta_u: 106, xi_u: 241, eta_v: 105, xi_v: 242, eta_psi: 105, xi_psi: 241, ocean_time: 1)
Coordinates:
  * s_rho           (s_rho) float64 -0.9875 -0.9625 -0.9375 ... -0.0375 -0.0125
  * s_w             (s_w) float64 -1.0 -0.975 -0.95 -0.925 ... -0.05 -0.025 0.0
    lon_rho         (eta_rho, xi_rho) float64 ...
    lat_rho         (eta_rho, xi_rho) float64 ...
    lon_u           (eta_u, xi_u) float64 ...
    lat_u           (eta_u, xi_u) float64 ...
    lon_v           (eta_v, xi_v) float64 ...
    lat_v           (eta_v, xi_v) float64 ...
    lon_psi         (eta_psi, xi_psi) float64 ...
    lat_psi         (eta_psi, xi_psi) float64 ...
  * ocean_time      (ocean_time) datetime64[ns] 2024-02-01T12:00:00
Dimensions without coordinates: tracer, boundary, eta_rho, xi_rho, eta_u, xi_u, eta_v, xi_v, eta_psi, xi_psi
Data variables: (12/107)
    ntimes          int32 ...
    ndtfast         int32 ...
    dt              float64 ...
    dtfast          float64 ...
    dstart          datetime64[ns] ...
    shuffle         int32 ...
    ...              ...
    salt            (ocean_time, s_rho, eta_rho, xi_rho) float64 ...
    shflux          (ocean_time, eta_rho, xi_rho) float64 ...
    ssflux          (ocean_time, eta_rho, xi_rho) float64 ...
    swrad_daily     (ocean_time, eta_rho, xi_rho) float64 ...
    sustr           (ocean_time, eta_u, xi_u) float64 ...
    svstr           (ocean_time, eta_v, xi_v) float64 ...
Attributes: (12/35)
    file:                            doppio_avg_6605_0001.nc
    format:                          netCDF-4/HDF5 file
    Conventions:                     CF-1.4, SGRID-0.3
    type:                            ROMS/TOMS nonlinear model averages file
    title:                           DOPPIO
    var_info:                        ../Data/varinfo1040t_daily.dat
    ...                              ...
    compiler_flags:                  -fp-model precise -heap-arrays -ip -O3 -...
    tiling:                          004x004
    history:                         ROMS/TOMS, Version 3.9, Sunday - Februar...
    ana_file:                        ROMS/Functionals/ana_btflux.h
    CPP_options:                     DOPPIO, ADD_FSOBC, ADD_M2OBC, ANA_BSFLUX...
    DODS_EXTRA.Unlimited_Dimension:  ocean_time

In [26]:
import pandas as pd

## Initialize dataframe with lat/long
df = pd.read_excel('/Users/henrysun_1/Desktop/Duke/2024-2025/Summer 2024/fishics/data/q2a_sites.xlsx') 
# njsites.xls found in demo folder - modify to my own data (select points)

df.rename(columns={"Unnamed: 0":"objectID","Unnamed: 1":"Lon","Unnamed: 2":"Lat", "Unnamed: 3":"Station"},inplace=True) # Change the column names to Lon/Lat
df = df.iloc[1:]
df.round(4)
df.round({"Lon":4, "Lat":4})
Lon = df.Lon
Lat = df.Lat


df

,objectID,Lon,Lat,Station
1,1,-74.204407,39.183304,3
2,2,-74.207153,39.104489,5
3,3,-74.092545,39.272988,6
4,4,-74.050598,39.325799,7
5,5,-74.130249,39.215231,8
6,6,-74.053345,39.217359,10
7,7,-74.440613,39.217359,11
8,8,-74.286804,39.404367,14
9,9,-74.130249,39.550648,15
10,10,-74.004084,39.47223,16


In [27]:
## This is where I would use xarray to select the dates you would like so you can subset the data. 
## the variable name is ocean_time and you can even slice the time using something like this: https://docs.xarray.dev/en/v2023.09.0/user-guide/indexing.html (xr.sel)
## Then you can loop through the points to calculate depth in meters, pull out stratification (stratification is the gradient in temperature in a water column profile divided by the gradient in depth (or dT/dz)
## basically just change in temp/change in depth) and temperature for each location:
## REMEMBER TO CHANGE DS IN ALLPNTS FROM DS TO WHATEVER YOU RENAME YOUR TIME SLICED DATASET
# ds = ds.sel(ocean_time = '2024-02-01', method = 'nearest')
temp_final=[]
dtdz_final = []
salt_final=[]
# ds


In [28]:
for k in range(21): # we setup our loop here and use k as a placeholder for iterating through our however many points

    Allpnts = abs(ds.lat_rho-Lat[k])+abs(ds.lon_rho-Lon[k]) # I've taken Lat/Lon from the dataframe and put them in your equations, then use k to tell the equation which version of k to use
    a,b = np.unravel_index(Allpnts.argmin(),Allpnts.shape)
    Allpnts=ds.sel(eta_rho=a, xi_rho=b)
    
    if Allpnts.Vtransform == 1:
        Zo_rho = Allpnts.hc * (Allpnts.s_rho - Allpnts.Cs_r) + Allpnts.Cs_r * Allpnts.h
        z_rho = Zo_rho + Allpnts.zeta * (1 + Zo_rho/Allpnts.h)
    elif Allpnts.Vtransform == 2:
        Zo_rho = (Allpnts.hc * Allpnts.s_rho + Allpnts.Cs_r * Allpnts.h) / (Allpnts.hc + Allpnts.h)
        z_rho = Allpnts.zeta + (Allpnts.zeta + Allpnts.h) * Zo_rho
   
    
    Allpnts.coords['z_rho'] = z_rho.transpose()
   
    temp = Allpnts.temp
    salt = Allpnts.salt
        
    temp_diff = np.diff(temp)/np.diff(np.abs(z_rho)) # Need to take the maximum of this to get max(dT/dz)
    # salt_diff=np.diff(salt)/np.diff(z_rho)
    
    tempdtdz1=np.max(-1*temp_diff, axis=1)
    # saltdtdz1=np.max(-1.*salt_diff, axis=1)

    #bottom_temp = temp[:,0]
    
    # plt.plot(tm,temp[:,39])
    # plt.plot(tm,bottom_temp)

# I did it!
    data_xr = xr.DataArray(tempdtdz1, 
    coords={'ocean_time': ds.ocean_time}, 
    dims=['ocean_time'])
    
    
    temp_final.append(temp) #this is an example of how you create a new xarray object, note that above I initialized temp_final as an empty array, and below I use the xarray concat function OUTSIDE of the loop... I don't know why this is how it works :) 
    # bottom_temp_final.append(bottom_temp)
    salt_final.append(salt)
    dtdz_final.append(data_xr)
    

## Now you can call this xarray object for temperature to plot temperature at a specific point. You may not want this one, but it should work the same with the other variables
combined_tempdtdz = xr.concat(dtdz_final,dim='k')
combined_temp=xr.concat(temp_final,dim='k')
combined_salt=xr.concat(salt_final,dim='k')
## now you can call combined_tempdtdz etc to make plots and look at data! or take averages over a time period. 
## These should have z_rho in their coordinates for the depth but let me know if that isn't true. 

KeyError: 0